# Feature Engineering Homework 
***
**Name**: $<$Rakesh Shivanand Margoor$>$ 

**Kaggle Username**: $<$Rakesh Margoor$>$
***

This assignment is due on Moodle by **5pm on Friday February 23rd**. Additionally, you must make at least one submission to the **Kaggle** competition before it closes at **4:59pm on Friday February 23rd**. Submit only this Jupyter notebook to Moodle. Do not compress it using tar, rar, zip, etc. Your solutions to analysis questions should be done in Markdown directly below the associated question.  Remember that you are encouraged to discuss the problems with your instructors and classmates, but **you must write all code and solutions on your own**.  For a refresher on the course **Collaboration Policy** click [here](https://github.com/chrisketelsen/CSCI5622-Machine-Learning/blob/master/resources/syllabus.md#collaboration-policy)



## Overview 
***

When people are discussing popular media, there’s a concept of spoilers. That is, critical information about the plot of a TV show, book, or movie that “ruins” the experience for people who haven’t read / seen it yet.

The goal of this assignment is to do text classification on forum posts from the website [tvtropes.org](http://tvtropes.org/), to predict whether a post is a spoiler or not. We'll be using the logistic regression classifier provided by sklearn.

Unlike previous assignments, the code provided with this assignment has all of the functionality required. Your job is to make the functionality better by improving the features the code uses for text classification.

**NOTE**: Because the goal of this assignment is feature engineering, not classification algorithms, you may not change the underlying algorithm or it's parameters

This assignment is structured in a way that approximates how classification works in the real world: Features are typically underspecified (or not specified at all). You, the data digger, have to articulate the features you need. You then compete against others to provide useful predictions.

It may seem straightforward, but do not start this at the last minute. There are often many things that go wrong in testing out features, and you'll want to make sure your features work well once you've found them.


## Kaggle In-Class Competition 
***

In addition to turning in this notebook on Moodle, you'll also need to submit your predictions on Kaggle, an online tournament site for machine learning competitions. The competition page can be found here:  

[https://www.kaggle.com/c/feature-engineering-csci-5622-spring-2018](https://www.kaggle.com/c/feature-engineering-csci-5622-spring-2018)

Additionally, a private invite link for the competition has been posted to Piazza. 

The starter code below has a `model_predict` method which produces a two column CSV file that is correctly formatted for Kaggle (predictions.csv). It should have the example Id as the first column and the prediction (`True` or `False`) as the second column. If you change this format your submissions will be scored as zero accuracy on Kaggle. 

**Note**: You may only submit **THREE** predictions to Kaggle per day.  Instead of using the public leaderboard as your sole evaluation processes, it is highly recommended that you perform local evaluation using a validation set or cross-validation. 

In [29]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
%matplotlib inline 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import FeatureUnion, Pipeline
import nltk
import numpy as np 
from scipy.sparse import csr_matrix
import scipy.sparse as sp
from sklearn.base import BaseEstimator, TransformerMixin
import re
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split


### [25 points] Problem 1: Feature Engineering 
***

The `FeatEngr` class is where the magic happens.  In it's current form it will read in the training data and vectorize it using simple Bag-of-Words.  It then trains a model and makes predictions.  

25 points of your grade will be generated from your performance on the the classification competition on Kaggle. The performance will be evaluated on accuracy on the held-out test set. Half of the test set is used to evaluate accuracy on the public leaderboard.  The other half of the test set is used to evaluate accuracy on the private leaderboard (which you will not be able to see until the close of the competition). 

You should be able to significantly improve on the baseline system (i.e. the predictions made by the starter code we've provided) as reported by the Kaggle system.  Additionally, the top **THREE** students from the **PRIVATE** leaderboard at the end of the contest will receive 5 extra credit points towards their Problem 1 score.


In [30]:
from sklearn.base import BaseEstimator, TransformerMixin

class LengthTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass
    
    def fit(self, examples):
        # return self and nothing else 
        return self
    
    def transform(self, examples):         
        # Initiaize matrix 
        X = np.zeros((len(examples), 1))
        
        # Loop over examples and count letters 
        for ii, x in enumerate(examples):
            X[ii,:] = np.array([len(x)])
            
        return csr_matrix(X) 

In [31]:
class SpecialCharCounter(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass
    
    def fit(self, examples):
        # return self and nothing else 
        return self
    
    def transform(self, examples): 
        specialChars = ['(', ')', ' ','  ','   ', ':' ]
        
        # Initiaize matrix 
        self.X = np.zeros((len(examples), len(specialChars)))
        
        # Loop over examples and count letters 
        for ii, x in enumerate(examples):
            self.X[ii,:] = np.array([x.count(specialChar) for specialChar in specialChars])
            
        return csr_matrix(self.X) 

In [32]:
from sklearn.base import BaseEstimator, TransformerMixin

class POSVerbTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass
    
    def fit(self, examples):
        # return self and nothing else 
        return self
    
    def transform(self, examples):
        
        import numpy as np 
        from scipy.sparse import csr_matrix
        
        verbs =["VBD", "VBG", "VBN","VBP","VBZ"] 
         
        # Initiaize matrix 
        X = np.zeros((len(examples), len(verbs)))
        
        # Loop over examples and count letters 
        for ii, x in enumerate(examples):
            X[ii,:] = np.array([x.count(verb) for verb in verbs])
        
        return csr_matrix(X) 


In [33]:
# Standard LemmaTokenizer
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]
def removepunc(text):
    punctuation = re.compile(r'[,.?!\':\)\(`";|0-9]')
    text = punctuation.sub('', text)
    return text

In [34]:
class FeatEngr:
    def __init__(self):
        
        from sklearn.feature_extraction.text import CountVectorizer
        from sklearn.pipeline import FeatureUnion
        
        self.totalfeatures = FeatureUnion([
            ('tfidf', Pipeline([
                ('tfidfer', TfidfVectorizer(analyzer = 'word',preprocessor = removepunc, ngram_range=(1,3),
                                stop_words = 'english', tokenizer = LemmaTokenizer()))
            ])),
            ('sentencelength', Pipeline([
                ('length',LengthTransformer())
            ])),
            ('specialchars', Pipeline([
                ('specialchar',SpecialCharCounter())
            ])),
        ])
        
    def fetchPOS(self, examples):
        mylist =[]
        for sen in examples:
            POS = (nltk.pos_tag(nltk.word_tokenize(sen)))
            senpos = ' '.join([p[1] for p in POS])
            mylist.append((senpos))
            senpos =''
        return mylist

    def build_train_features(self, examples):
        """
        Method to take in training text features and do further feature engineering 
        Most of the work in this homework will go here, or in similar functions  
        :param examples: currently just a list of forum posts  
        """        
        
        self.postransform = POSVerbTransformer()
        pos = self.fetchPOS(examples)
        posfeatures = self.postransform.fit_transform(pos)
        
        basefeatures = self.totalfeatures.fit_transform(examples)
        return  sp.hstack([basefeatures, posfeatures], format ='csr')

    def get_test_features(self, examples):
        """
        Method to take in test text features and transform the same way as train features 
        :param examples: currently just a list of forum posts  
        """
        pos = self.fetchPOS(examples)
        posfeatures = self.postransform.transform(pos)
    
        basefeatures = self.totalfeatures.transform(examples)
        #print((posfeatures.shape))
        #print((basefeatures.shape))
        return  sp.hstack([basefeatures, posfeatures], format ='csr')

    def show_top10(self):
        """
        prints the top 10 features for the positive class and the 
        top 10 features for the negative class. 
        """
        feature_names = np.asarray(self.allmyfeatures.get_feature_names())
        top10 = np.argsort(self.logreg.coef_[0])[-10:]
        bottom10 = np.argsort(self.logreg.coef_[0])[:10]
        print("Pos: %s" % " ".join(feature_names[top10]))
        print("Neg: %s" % " ".join(feature_names[bottom10]))
                
    def train_model(self, random_state=1234):
        """
        Method to read in training data from file, and 
        train Logistic Regression classifier. 
        
        :param random_state: seed for random number generator 
        """
        
        from sklearn.linear_model import LogisticRegression 
        
        # load data 
        dfTrain = pd.read_csv("../data/spoilers/train.csv").to_dict('list')   
        
        #self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(list(dfTrain["sentence"]), np.array(dfTrain["spoiler"], dtype=int), test_size=0.2, random_state=10)
        
        # get training features and labels 
        self.X_train = self.build_train_features(list(dfTrain["sentence"]))
        self.y_train = np.array(dfTrain['spoiler'], dtype=int)
        
        
        # train logistic regression model.  !!You MAY NOT CHANGE THIS!! 
        self.logreg = LogisticRegression(random_state=random_state)
        self.logreg.fit(self.X_train, self.y_train)
        
        
        
        
    def model_predict(self):
        """
        Method to read in test data from file, make predictions
        using trained model, and dump results to file 
        """
        
        # read in test data 
        dfTest  = pd.read_csv("../data/spoilers/test.csv").to_dict('list')
        
        # featurize test data 
        self.X_test = self.get_test_features(list(dfTest["sentence"]))
        self.ErrorX = self.X_test
        #self.X_test = self.get_test_features(self.X_test)
        scores = cross_val_score(self.logreg, self.X_train, self.y_train, cv =10)
        print(scores.mean(), scores.std())
        
        # make predictions on test data 
        pred = self.logreg.predict(self.X_test)
        
        ################ Error Analysis###########################
        #totalCount = 0
#       trueCount = 0
#       index =0
#       for x, y in zip(pred, self.y_test):
#           if x !=y:
#               print(self.ErrorX[index])
#           index  += 1   
        
        # dump predictions to file for submission to Kaggle  
        pd.DataFrame({"spoiler": np.array(pred, dtype=bool)}).to_csv("prediction.csv", index=True, index_label="Id")

In [35]:
# Instantiate the FeatEngr clas 
feat = FeatEngr()

# Train your Logistic Regression classifier 
feat.train_model(random_state=1230)

# Shows the top 10 features for each class 
#feat.show_top10()

# Make prediction on test data and produce Kaggle submission file 
feat.model_predict()

0.692899667273 0.0172030569455


### [25 points] Problem 2: Motivation and Analysis 
***

The job of the written portion of the homework is to convince the grader that:

- Your new features work
- You understand what the new features are doing
- You had a clear methodology for incorporating the new features

Make sure that you have examples and quantitative evidence that your features are working well. Be sure to explain how you used the data (e.g., did you have a validation set? did you do cross-validation?) and how you inspected the results. In addition, it is very important that you show some kind of an **error analysis** throughout your process.  That is, you should demonstrate that you've looked at misclassified examples and put thought into how you can craft new features to improve your model. 

A sure way of getting a low grade is simply listing what you tried and reporting the Kaggle score for each. You are expected to pay more attention to what is going on with the data and take a data-driven approach to feature engineering.

Some of the features that I used and tried to improve the accuracy of finding whether a sentence is spoiler or not is mentioned below.
To analyse the accuracy of the model, I used cross validation on the training set through cross_val_score function from sklearn with 10 folds. To perform the error analyis, I used the train_test_split for 80% to 20% ratio. 

With all the below mentioned features, I was able to get an highest accuracy of ***0.68817*** at Kaggle.

### TfIdf feature. 

In the given large training set, some words are present such as “the”, “a”, “is” which carry very little meaningful information about the actual contents of the document. 
If we could feed the direct count data directly to a classifier those very frequent terms would shadow the frequencies of rarer yet more interesting terms.
Hence, this was a better feature than the normal countVectorizer()

The increase in cross_val_score between the normal bag-of-words (CountVectorizer) to tf-idf features is 2.10 %.

### N-grams
Bag of words does not consider the order of occurrence of the words. Hence, some of the features are lost. n-gram is something which considers the order of occurrences and hence would create an extra feature space. I restricted n to 3 considering only bi-grams and tri-grams, as the occurrence of 4 words simultaniously in mulitple documents is less. With bigram and tri-gram, I was able to improve my accuracy by 0.9%.

### POS(Part of Speech) Tagging:
I used the POS feature because when I went through the training data, I was able to see more verbs which came after nouns, which indicates, if a person is saying something about spoiler he has to refer a person or the character followed by an action that took place in the movie/event. So I guessed, having bi-gram features of the POS tags of words was an interesting feature.
After some error analyis, I saw that there were non-spoiler statements which had similar bi-gram POS tagging features as well. 
Hence, I used the bag-of-words of the verbs occuring i.e the more the verbs, the more probabilty of predicting it as a spoiler. But it did not completely boost the accuracy as many of the non-spoilers had a slightly lesser verbs. For example,  one of the misclassified example was:

[:  Cobb first appears to be killing randomly, until Rebecca figures out that he kills random people in order to conceal his real targets:   15-16 year old girls, like the sister he was resentful of because their mom took care of her but abandoned him at an orphanage.]
        whose POS tagging was

[': NN RB VBZ TO VB VBG RB , IN NNP VBZ RP IN PRP VBZ JJ NNS IN NN TO VB PRP$ JJ NNS : JJ NN JJ NNS , IN DT NN PRP VBD NN IN IN PRP$ NN VBD NN IN PRP$ CC VBD PRP IN DT NN .']

As you can see, there are more than 8 verbs, and there are nouns occuring before verb, yet is considered as non-spoiler.
After careful analyis, I think we have to appropriately stem the sentences and consider the verb of the third person which can be extracted by NLTK tags. This might increase our accuracy, which I havent tried in this assignment.
Through this feature, I was able to improve the accuracy only by 0.5%.


### Special Characters

Special Characters is something which was quite tough to explore. I observed that the spoilers had more number of special characters, especially double quotes and brackets. Digging into the data, I found that, the instances such as places, names were being sorrounded by double quotes, which are ususally used when trying to refer something, making it more probaable towards spoiler.

Some of the examples to justify the above are:

-  Animal Motifs One of Sydney's call signs   after she returned from the "dead"  was  "Phoenix" ; Jack's call signs are all birds ("Blackbird" with SD-6, "Raptor" with the CIA).

-  In the flashback in "Home Invasion," the nurse who is referred to as a "lezzie" is drowned in the bathtub later that same scene.

Incorporating these features, my accuracy increased by 1.3%

### Sentence length or number of words in sentence

Length of a sentence is one of the feature that I explored analysing the train data. I performed a thoruough analysis on how the length of postive and the negative spoilers were. Unfortunately there was not much of a big difference as you can see in the below code where only around an additional of 800 positive samples were having a higher average length than the negative samples. Because of this, there was not a big improvement in the accuracy,  and it resulted in a improvement of 1%

In [36]:
dfTest  = pd.read_csv("../data/spoilers/train.csv")
sentences = list(dfTest["sentence"]) 
isSpoilers = list(dfTest["spoiler"]) 
totalpos =0
totalposlen =0
totalneglen =0
totalneg =0 

for i, j in zip(sentences, isSpoilers):
    if j == True:
        totalpos += 1
        totalposlen += len(i)
    else:
        totalneg += 1
        totalneglen += len(i)

print("Positive Average length =",(totalposlen/totalpos), "Negative Average length =",(totalneglen/totalneg))

totalavg = ((totalposlen/totalpos) + (totalneglen/totalneg))/2
posCount =0 
negCount =0
for i, j in zip(sentences, isSpoilers):
    if len(i) > totalavg:
        if j == True:
            posCount += 1
        else:
            negCount += 1

print("Positive with length above average =",posCount,"Negatives with length above average  =", negCount)

Positive Average length = 120.85846055979644 Negative Average length = 104.15047518479409
Positive with length above average = 2892 Negatives with length above average  = 2086


### Hints 
***

- Don't use all the data until you're ready. 

- Examine the features that are being used.

- Do error analyses.

- If you have questions that aren’t answered in this list, feel free to ask them on Piazza.

### FAQs 
***

> Can I heavily modify the FeatEngr class? 

Totally.  This was just a starting point.  The only thing you cannot modify is the LogisticRegression classifier.  

> Can I look at TV Tropes?

In order to gain insight about the data yes, however, your feature extraction cannot use any additional data (beyond what I've given you) from the TV Tropes webpage.

> Can I use IMDB, Wikipedia, or a dictionary?

Yes, but you are not required to. So long as your features are fully automated, they can use any dataset other than TV Tropes. Be careful, however, that your dataset does not somehow include TV Tropes (e.g. using all webpages indexed by Google will likely include TV Tropes).

> Can I combine features?

Yes, and you probably should. This will likely be quite effective.

> Can I use Mechanical Turk?

That is not fully automatic, so no. You should be able to run your feature extraction without any human intervention. If you want to collect data from Mechanical Turk to train a classifier that you can then use to generate your features, that is fine. (But that’s way too much work for this assignment.)

> Can I use a Neural Network to automatically generate derived features? 

No. This assignment is about your ability to extract meaningful features from the data using your own experimentation and experience.

> What sort of improvement is “good” or “enough”?

If you have 10-15% improvement over the baseline (on the Public Leaderboard) with your features, that’s more than sufficient. If you fail to get that improvement but have tried reasonable features, that satisfies the requirements of assignment. However, the extra credit for “winning” the class competition depends on the performance of other students.

> Where do I start?  

It might be a good idea to look at the in-class notebook associated with the Feature Engineering lecture where we did similar experiments. 


> Can I use late days on this assignment? 

You can use late days for the write-up submission, but the Kaggle competition closes at **4:59pm on Friday February 23rd**

> Why does it say that the competition ends at 11:59pm when the assignment says 4:59pm? 

The end time/date are in UTC.  11:59pm UTC is equivalent to 4:59pm MST.  Kaggle In-Class does not allow us to change this. 